In [1]:
#Условие:
# Input_size = 300 
# Activation Function = Sigmoid -> исходя из этого условия, мы можем утверждать, что наша задача сводится к построению логистической регрессии, реализовывающую бинарную классификацию 


# Логистическая регрессия состоит из линейной регрессии (линейного преобразования) + функции активации
# На нашем примере: синтез 300 линейных преобразований и одной функции активации (sigmoid), вносящую нелинейное преобразование (чтобы моделька выявила сложные паттерны/зависимости и обучалась на них)


# Создаём модель ;-)

import numpy as np
from numba import jit
import timeit
from sklearn.metrics import classification_report, confusion_matrix

class LogisticRegression:
    def __init__(self, input_dim, learning_rate=0.01): #input_dim - число входных признаков, при вызове обозначим 300, как сказано в условии
        self.weights = np.random.rand(input_dim) #300 раз генерирует случайные числа из равномерного распределения в диапазоне от 0 до 1, тем самым имитируя инициализацию весами для каждого входа
        self.bias = np.random.rand() #аналогично делает для смещений, имитируя инициализацию
        self.learning_rate = learning_rate #коэффициент "небрежности" при обновлении весов на основе градиента, чем меньше - тем плавнее мы будем обновлять веса

    def sigmoid(self, z): # создаём нашу функцию активации
        return 1 / (1 + np.exp(-z)) #её формула 
    
    
    def predict_dot(self, X): 
        z = np.dot(X, self.weights) + self.bias # реализуем линейное преобразование
        return self.sigmoid(z) # посылаем результат линейного преобразования в функцию активации, чтобы осуществить предсказание вероятности принадлежности к классу 
    
    
    #версия 2, ручное умножение матриц + ускорение с jit
    @jit
    def predict_jit(self, X):
        y_pred = np.zeros(X.shape[0])
        for i in range(X.shape[0]):  # Итерация по всем строкам в X
            z = 0
            for j in range(X.shape[1]): # Итерация по всем элементам в строке
                z += X[i, j] * self.weights[j]  # Покоординатное умножение и суммирование
            z += self.bias
            y_pred[i] = self.sigmoid(z)  # Применение сигмоидной функции
        return y_pred
        

    def loss(self, y_true, y_pred): #приниманием истинное и предсказанное 
        return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred)) #возвращаем потерю в результате нашего предсказание

    def gradient_descent(self, X, y_true, y_pred):
    # Ниже мы будем имитировать градиетный спуск, опишу зачем: 
    
    # Мы получаем значения потерь и должны задаться следующим вопросом: 
    # «Какова механика изменения функции, в результате изменения её аргумента и чем она характеризуется?»
    # На языке математики: «Найди скорость приращения функции, при приращении её аргумента»
    # На языке практики это: «Какова механика изменения потерь, в результате изменения весов?»
    
    # Ответить на данный вопрос можно только после вычисления производной (при помощи Backpropagation)
    # Результат которой является синтезом двух компонент (то она чем характеризуется)
    
    # Компонента 1 - Направление изменения функции (её секущая) ~ tg a
    # Если tg a > 0 => функция растет / Если tg a < 0 => функция убывает
    
    # Компонента 2 - Скорость изменения функции в моменте (касательная)

        m = len(y_true) # Давай Получаем количество обучающих примеров
    
        # Давай вычислим градиенты (производные) потерь по весам (dW) и смещению (dB)
        dw = np.dot(X.T, (y_pred - y_true)) / m # Градиент по весам
        db = np.sum(y_pred - y_true) / m # Градиент по смещению
    
        # Давай опишем как будем обновлять параметры модельки, на основе проделанных действий
        self.weights -= self.learning_rate * dw # Обновляем веса (с учетом информации о градиенте)
        self.bias -= self.learning_rate * db # Обновляем смещения (с учетом информации о градиенте)
        
    
    def gradient_descent_jit(self, X, y_true, y_pred): #покоординатное умножение элементов матрицы "руками"
        m = len(y_true)
        
        dw = np.zeros_like(self.weights)
        db = 0
        
        #вычисляем градиенты
        for i in range(m):
            for j in range(X.shape[1]):
                dw[j] += (y_pred[i] - y_true[i]) * X[i, j]
            db += (y_pred[i] - y_true[i])
        
        dw /= m
        db /= m
        
        #Обновляем веса и смещение
        self.weights -= self.learning_rate * dw
        self.bias -= self.learning_rate * db
                

    def fit_dot(self, X, y, epochs=500): #описываем алгоритм тренировочного цикла в виде вызываемой функции
        for epoch in range(epochs): # 500 эпох = 500 раз проходимся по всему датасету, в процессе обучения модели
            y_pred = self.predict_dot(X) # Вычисление предсказаний модели на текущем наборе данных
            y_pred_labels = (y_pred > 0.5).astype(int) #преобразуем вероятности в метки посредством округления
            self.gradient_descent(X, y, y_pred) # Запуск обновление весов и смещения модели с использованием градиентного спуска
            if epoch % 10 == 0: # Вывод информации о процессе обучения каждую эпоху
                print(f"Epoch {epoch}: Loss = {self.loss(y, y_pred)}")
            
        # Оценим качество модели на сгенерированном синтетическом датасете:
        # Вывод результатов
        report = classification_report(y, y_pred_labels)
        conf_matrix = confusion_matrix(y, y_pred_labels)
            
        return report, conf_matrix
    
        print("Classification Report:")
        print(report)
        print("Confusion Matrix:")
        print(conf_matrix)
        
    
        
    def fit_jit(self, X, y, epochs=500): #описываем алгоритм тренировочного цикла в виде вызываемой функции
        for epoch in range(epochs): # 500 эпох = 500 раз проходимся по всему датасету, в процессе обучения модели
            y_pred = self.predict_jit(X) # Вычисление предсказаний модели на текущем наборе данных
            y_pred_labels = (y_pred > 0.5).astype(int) #преобразуем вероятности в метки посредством округления
            self.gradient_descent_jit(X, y, y_pred) # Обновление весов и смещения модели с использованием градиентного спуска
            if epoch % 10 == 0: # Вывод информации о процессе обучения каждую эпоху
                print(f"Epoch {epoch}: Loss = {self.loss(y, y_pred)}")
                
        # Оценим качество модели на сгенерированном синтетическом датасете:
        # Вывод результатов
        report = classification_report(y, y_pred_labels)
        conf_matrix = confusion_matrix(y, y_pred_labels)
            
        return report, conf_matrix
        
        print("Classification Report:")
        print(report)
        print("Confusion Matrix:")
        print(conf_matrix)

/var/folders/gv/nkdtfy695vx8nny1jj5gqygm0000gn/T/ipykernel_1491/1656287518.py:33: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit


In [2]:
# Давай используем, но прежде нужно отправить подготовленный датасет в нашу модельку

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Создаем синтетический датасет
X, y = make_classification(n_samples=1000, n_features=300, n_classes=2, random_state=42)

# Разделяем данные на тренировочный и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Приводим наши данные к единому стандарту
# чтобы каждый признак (или столбец данных) имел среднее значение (математическое ожидание) 0 
# А стандартное отклонение было равно 1. 
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test) 

# Создание экземпляра модели
model = LogisticRegression(input_dim=300) 

#Обучаем модельки и выводим метрики оценки качества
report_dot, conf_matrix_dot = model.fit_dot(X_train_scaled, y_train) #вызов обучения по dot
print("Report (dot):")
print(report_dot)
print("Confusion Matrix (dot):")
print(conf_matrix_dot)

report_jit, conf_matrix_jit = model.fit_jit(X_train_scaled, y_train) #вызов обучения по jit
print("Report (jit):")
print(report_jit)
print("Confusion Matrix (jit):")
print(conf_matrix_jit)

#Вызываем 
prediction_using_dot = model.predict_dot(X_train_scaled)  # Вызов метода predict_dot
prediction_using_jit = model.predict_jit(X_train_scaled)  # Вызов метода predict_jit


# Функции для замера времени
def time_predict_dot():
    model.predict_dot(X_train_scaled) # Вызов метода predict_dot

def time_predict_jit():
    model.predict_jit(X_train_scaled) # Вызов метода predict_jit

# Измеряем время
time_dot = timeit.timeit(time_predict_dot, number=10)
time_jit = timeit.timeit(time_predict_jit, number=10)

# Операция выполняется 10 раз для получения более надежной оценки времени выполнения кода. 
# При многократном выполнении кода усредняется время выполнения, что помогает уменьшить влияние случайных задержек, 
# которые могут возникать в системе (например, другие процессы, работающие на компьютере, влияние кэширования и т.д.).

print(f"Время выполнения с np.dot: {time_dot} секунд")
print(f"Время выполнения с numba.jit: {time_jit} секунд")

Epoch 0: Loss = 4.256383888301372
Epoch 10: Loss = 4.189481939116288
Epoch 20: Loss = 4.123704538196315
Epoch 30: Loss = 4.059033249043813
Epoch 40: Loss = 3.995448379464272
Epoch 50: Loss = 3.93292917639433
Epoch 60: Loss = 3.8714547994997814
Epoch 70: Loss = 3.8110050015955546
Epoch 80: Loss = 3.7515608017159208
Epoch 90: Loss = 3.693104937935982
Epoch 100: Loss = 3.6356223055034014
Epoch 110: Loss = 3.5790999795701772
Epoch 120: Loss = 3.5235272477747634
Epoch 130: Loss = 3.468895170370713
Epoch 140: Loss = 3.4151957749422257
Epoch 150: Loss = 3.362420929215047
Epoch 160: Loss = 3.3105609674934384
Epoch 170: Loss = 3.2596036315350507
Epoch 180: Loss = 3.209533759835291
Epoch 190: Loss = 3.1603338536446652
Epoch 200: Loss = 3.111985314687546
Epoch 210: Loss = 3.0644698579061087
Epoch 220: Loss = 3.0177707255513417
Epoch 230: Loss = 2.9718733848738417
Epoch 240: Loss = 2.9267658390128464
Epoch 250: Loss = 2.8824385510425725
Epoch 260: Loss = 2.8388841477381708
Epoch 270: Loss = 2.7960

/var/folders/gv/nkdtfy695vx8nny1jj5gqygm0000gn/T/ipykernel_1491/1656287518.py:33: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "predict_jit" failed type inference due to: non-precise type pyobject
During: typing of argument at /var/folders/gv/nkdtfy695vx8nny1jj5gqygm0000gn/T/ipykernel_1491/1656287518.py (33)

File "../../../../var/folders/gv/nkdtfy695vx8nny1jj5gqygm0000gn/T/ipykernel_1491/1656287518.py", line 33:
<source missing, REPL/exec in use?>

  @jit
/var/folders/gv/nkdtfy695vx8nny1jj5gqygm0000gn/T/ipykernel_1491/1656287518.py:33: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "predict_jit" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "../../../../var/folders/gv/nkdtfy695vx8nny1jj5gqygm0000gn/T/ipykernel_1491/1656287518.py", line 36:
<source missing, REPL/exec in use?>

  @jit
/Users/magomedgazaev/Deep Lear

Epoch 0: Loss = 2.000774627996841
Epoch 10: Loss = 1.9730951475924048
Epoch 20: Loss = 1.9458940351766336
Epoch 30: Loss = 1.9191659682647326
Epoch 40: Loss = 1.8929068994766567
Epoch 50: Loss = 1.8671137377804052
Epoch 60: Loss = 1.8417839794957696
Epoch 70: Loss = 1.8169153384030279
Epoch 80: Loss = 1.7925054162087497
Epoch 90: Loss = 1.768551438535489
Epoch 100: Loss = 1.7450500794397226
Epoch 110: Loss = 1.7219973707352687
Epoch 120: Loss = 1.6993886838354673
Epoch 130: Loss = 1.6772187632188615
Epoch 140: Loss = 1.6554817880210897
Epoch 150: Loss = 1.6341714360974715
Epoch 160: Loss = 1.613280938950527
Epoch 170: Loss = 1.5928031157574185
Epoch 180: Loss = 1.5727303862807311
Epoch 190: Loss = 1.5530547703021853
Epoch 200: Loss = 1.5337678861644826
Epoch 210: Loss = 1.5148609615142776
Epoch 220: Loss = 1.4963248679568728
Epoch 230: Loss = 1.4781501867857145
Epoch 240: Loss = 1.4603273033493536
Epoch 250: Loss = 1.4428465209937469
Epoch 260: Loss = 1.4256981846894203
Epoch 270: Loss

In [ ]:
#Время выполнения с np.dot: 0.00027233303990215063 секунд
#Время выполнения с numba.jit: 0.473667333018966 секунд